# MakerSpace Jam

In [1]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Primary Dependencies and Context Setting

In [2]:
!pip3 install -U -q openai==0.27.8 llama-index==0.8.6 nltk==3.8.1 python-dotenv

### Load the OPENAI API key

In [3]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

### Load token list

In [4]:
import pandas as pd

sheet_name = "GSW_token_list"
workbook_id = '1MB1ZsQul4AB262AsaY4fHtGW4HWp2-56zB-E5xTbs2A'
url = f'https://docs.google.com/spreadsheets/d/{workbook_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df = pd.read_csv(url)
df.head()

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


,Staff,Job title,Token
0,Klay Thompson,Player,1bd8c9ee72e314e1dd8b2442e568fc3d
1,Jonathan Kuminga,Player,1bd8c9ee72e314e1dd8b2442e568fc3d
2,Chris Paul,Player,1bd8c9ee72e314e1dd8b2442e568fc3d
3,Stephen Curry,Player,1bd8c9ee72e314e1dd8b2442e568fc3d
4,Moses Moody,Player,1bd8c9ee72e314e1dd8b2442e568fc3d


In [5]:
token_list = df['Token'].tolist()

In [30]:
token_string = '_'.join(token_list)
token_string

'1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee72e314e1dd8b2442e568fc3d_1bd8c9ee7

## Data Loading

### Webpage

Using the Trafilatura Web Reader.

In [6]:
!pip3 install trafilatura

In [7]:
# list of webpages to index
webpages = [ "https://www.basketball-reference.com/" ]

In [8]:
from llama_index import TrafilaturaWebReader

web_docs = TrafilaturaWebReader().load_data([webpages[0]])
web_docs[0].text

"Basketball Stats and History Statistics, scores, and history for the NBA, ABA, WNBA, and top European competition.\nEvery NBA & Every WNBA Player\nPlay Immaculate Grid\nPut your men's basketball knowledge to the test with our daily men's basketball trivia game. Can you complete the grid?\nRecent DebutsChance Comanche (POR), Jacob Gilyard (MEM), RaiQuan Gray (BRK), Justin Minaya (POR), Donovan Williams (ATL) and Jeenathan Williams (POR)\nNBA & ABA, WNBA, NBL, G League, and top International players\nIncludes indexed lists of players. International leagues include top European leagues and EuroLeague and EuroCup competitions, as well as China's CBA, Australia's NBL, and Men's Olympics.\nIn MemoriamHenry Dickerson, Charles Balentine, George Wilson, Henry Logan, Dedric Willoughby and Nikki McCray\nPlayer pages include basic statistics and links to player's game logs, splits, advanced stats, and more.\nEvery NBA Team\n2023-24 NBA Standings\nSummary · Schedule · Scores · Leaders · Rookies\nT

In [9]:
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(
    chunk_size=1000 ### YOUR CODE HERE
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter ### YOUR CODE HERE
)

# parse nodes from workbooks and insert into vector index
for w_doc in web_docs:
    nodes = node_parser.get_nodes_from_documents([w_doc])
    for node in nodes:
        node.metadata = {'title': 'Basketball Stats and History',
                         'type': 'webpage',
                         'url': 'https://www.basketball-reference.com/',
                         'description': 'Basketball Stats and History',
                         'accessibility': 'public'
                        }

### Email Samples

In [13]:
from llama_index import Document
from llama_index.schema import MetadataMode

In [96]:
docEmailSample = Document(
    text="Hey KD, let's grab dinner after the game on October 24th, Steph", 
    metadata={
        'doctype': 'email',
        # 'datetime': "2023-09-01T15:30:00Z",
        # 'email_from': 'kevin.durant@suns.nba',
        # 'email_to': 'stephen.curry@warriors.nba',
        # "token_list": token_string,
        'from': 'Stephen Curry',
        'to': 'Kevin Durant'
    }
)
docEmailSample2 = Document(
    text="Yo, you were a monster last year, can't wait to play against you in the opener! Draymond", 
    metadata={
        'doctype': 'email',
        # 'datetime': "2023-09-01T02:30:00Z",
        # 'email_from': 'lebron.james@lakers.nba',
        # 'email_to': 'nikola.jokic@nuggets.nba',
        # "token_list": token_string,
        # 'team': 'Golden State Warriors',
        'from': 'Draymond Green',
        'to': 'Nikola Jokic'
    }
)
docScheduleSample = Document(
    text="""
        {
            "Date": "Tue Oct 24 2023",
            "Start (ET)": "7:30p",
            "Visitor/Neutral": "Golden State Warriors",
            "PTS": "",
            "Home/Neutral": "Denver Nuggets",
            "PTS": "",
            "Attend.": "",
            "Arena": "Ball Arena",
            "Notes": ""
        }
    """,
    metadata={
        'doctype': 'schedule',
        # "token_list": token_string,
        'teams': 'Golden State Warriors' # idk how to get list of metadata
    }
)
docScheduleSample2 = Document(
    text="""
        {
            "Date": "Fri Oct 27 2023",
            "Start (ET)": "10:00p",
            "Visitor/Neutral": "Phoenix Suns",
            "PTS": "",
            "Home/Neutral": "Golden State Warriors",
            "PTS": "",
            "Attend.": "",
            "Arena": "Chase Center",
            "Notes": ""
        }
    """,
    metadata={
        'doctype': 'schedule',     
        # "token_list": token_string,
        'team': 'Golden State Warriors' # idk how to get list of metadata
############################ YOU ASK(??) FOR SMTH LIKE THIS:
        # 'list': [
        #     {'team': 'warriors'}, 
        #     {'team': 'lakers'},
        #     {'team': 'nuggets'},
        # ]
    }
)

docAdditionalSamples = [docEmailSample, docEmailSample2, docScheduleSample, docScheduleSample2]

### Example to play with

In [21]:
docPano = Document(
    text='NBA Player born Nov 1989 in Greece', 
    metadata={
        'type': 'player', 
        'number': '42', 
        'team': 'surfers', 
        'position': 'floating point guard',
        'specialty': 'eurostepping',
        'dress-style': 'barefoot',
        'secret': 'loves pizza'
    }
)
docChristos = Document(
    text='NBA Player born Dec 1994 in the USA', 
    metadata={
        'type': 'player', 
        'number': '3', 
        'team': 'climbers', 
        'position': 'strong forward pass',
        'specialty': 'ally-oop',
        'dress-style': 'pink-on-Wednesdays',
        'secret': 'loves cats, candy and cartoons'
    },
    excluded_llm_metadata_keys=['secret'],
    metadata_seperator="::",
    metadata_template="{key}=>{value}",
    text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
)

docExamplesToPlayWith = [docPano, docChristos]

print("The LLM sees this: \n", docChristos.get_content(metadata_mode=MetadataMode.LLM))
print("The Embedding model sees this: \n", docChristos.get_content(metadata_mode=MetadataMode.EMBED))

The LLM sees this: 
 Metadata: type=>player::number=>3::team=>climbers::position=>strong forward pass::specialty=>ally-oop::dress-style=>pink-on-Wednesdays
-----
Content: NBA Player born Dec 1994 in the USA
The Embedding model sees this: 
 Metadata: type=>player::number=>3::team=>climbers::position=>strong forward pass::specialty=>ally-oop::dress-style=>pink-on-Wednesdays::secret=>loves cats, candy and cartoons
-----
Content: NBA Player born Dec 1994 in the USA


## Indexing

### Context setting

In [22]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()
chunk_size = 1000
llm = OpenAI(
    temperature=0, 
    model="gpt-3.5-turbo",
    streaming=True
)

service_context = ServiceContext.from_defaults(
    llm=llm, 
    chunk_size=chunk_size,
    embed_model=embed_model
)

### Vector Store

This is intended to be a global vector store to insert the nodes from all data sources

In [23]:
!pip3 install -U -q chromadb==0.4.6 tiktoken==0.4.0 sentence-transformers==2.2.2 pydantic==1.10.11

In [24]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import chromadb

In [97]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("all_data")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context, service_context=service_context)

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


### Populate Vector Store with Nodes

Parse nodes for each loaded data source and insert it to the vector store.

In [98]:
# vector_index.insert_nodes(nodes) 
# vector_index.insert_nodes(docExamplesToPlayWith)
vector_index.insert_nodes(docAdditionalSamples)

### Setup Metadata Filtering

In [111]:
from llama_index.tools import FunctionTool
from llama_index.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

top_k = 3

info_emails_players = VectorStoreInfo(
    content_info="emails exchanged between NBA players",
    metadata_info=[
        MetadataInfo(
            name="from",
            type="str",
            description="""
email sent by a player of the Golden State Warriors, one of [Stephen Curry, 
Klay Thompson, 
Chris Paul, 
Andrew Wiggins,
Draymond Green,
Gary Payton II,
Kevon Looney,
Jonathan Kuminga,
Moses Moody,
Brandin Podziemski,
Cory Joseph,
Dario Šarić]"""
        ), 
        MetadataInfo(
            name="to",
            type="str",
            description="""receiver of the email text, can be any NBA player"""
        ), 
        MetadataInfo(
            name='doctype',
            type="str",
            description="email"
        )
    ]
)
info_schedule_team = VectorStoreInfo(
    content_info="schedule of the Golden State Warriors games",
    metadata_info=[
        MetadataInfo(
            name="team",
            type="str",
            description="Golden State Warriors"
        ), 
        MetadataInfo(
            name='doctype',
            type="str",
            description="schedule"
        )
    ]
)

In [112]:
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )

In [113]:
token = token_list[0]

In [114]:
def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"
    
    # for i, (k, v) in enumerate(zip(filter_key_list, filter_value_list)):
    #     if k == 'token_list':
    #         if token not in v:
    #             v = ''

    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        vector_index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    query_engine = RetrieverQueryEngine.from_args(retriever)

    response = query_engine.query(query)
    return str(response)

In [115]:
description_emails = f"""\
Use this tool to look up information about emails exchanged betweed players of the Golden State Warriors and any other NBA player.
The vector database schema is given below:
{info_emails_players.json()}
"""
auto_retrieve_tool_emails = FunctionTool.from_defaults(
    fn=auto_retrieve_fn, 
    name='auto_retrieve_tool_emails',
    description=description_emails, 
    fn_schema=AutoRetrieveModel
)

description_schedule = f"""\
Use this tool to look up the NBA game schedule of the Golden State Warriors.
The vector database schema is given below:
{info_schedule_team.json()}
"""
auto_retrieve_tool_schedule = FunctionTool.from_defaults(
    fn=auto_retrieve_fn, 
    name='auto_retrieve_tool_schedule',
    description=description_schedule, 
    fn_schema=AutoRetrieveModel
)



## Query Engine

In [36]:
query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    vector_store_query_mode="default",
    filters=MetadataFilters(
        filters=[
            ExactMatchFilter(key="team", value="warriors"),
            [ExactMatchFilter(key="token_list", value=token_string)],
             for v in token_list
        ]
    ),
    alpha=None,
    doc_ids=None,
    # chat_mode='react',
)

In [37]:
# response = query_engine.query("Which teams are we playing this month? Do I have any email communication indicating a meeting with any of the players of those teams")
response = query_engine.query("Which teams are we playing this month?")
print(response.response)
response.metadata

We are playing against the Phoenix Suns this month.


{'92593b92-8af4-46ed-9e13-1fd246f9dda0': {'type': 'email',
  'datetime': '2023-09-01T15:30:00Z',
  'email_from': 'kevin.durant@suns.nba',
  'email_to': 'stephen.curry@warriors.nba',
  'team': 'warriors'},
 '06a69ec8-3ba5-44c1-964f-837f04e4bf5f': {'type': 'schedule',
  'team': 'warriors'}}

## SQL NBA Query Tool

In [ ]:
!pip3 install -q -U pandas

In [74]:
import pandas as pd

def get_df_from_workbook(sheet_name,
                         workbook_id = '1MB1ZsQul4AB262AsaY4fHtGW4HWp2-56zB-E5xTbs2A'):
    url = f'https://docs.google.com/spreadsheets/d/{workbook_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    return pd.read_csv(url)

In [76]:
sheet_names = ['Teams', 'Players', 'Schedule', 'Player_Stats']
dict_of_dfs = {sheet: get_df_from_workbook(sheet) for sheet in sheet_names}
dict_of_dfs['Teams'].head()

,Franchise,Lg,From,To,Yrs,G,W,L,W/L%,Plyfs,Div,Conf,Champ
0,Atlanta Hawks,NBA,1949-50,2023-24,75,5855,2891,2964,0.494,49,12,0,1
1,Boston Celtics,NBA/BAA,1946-47,2023-24,78,6032,3570,2462,0.592,60,33,10,17
2,Brooklyn Nets,NBA/ABA,1967-68,2023-24,57,4530,1996,2534,0.441,31,5,2,2
3,Charlotte Hornets,NBA,1988-89,2023-24,34,2631,1153,1478,0.438,10,0,0,0
4,Chicago Bulls,NBA,1966-67,2023-24,58,4598,2344,2254,0.510,37,9,6,6


In [77]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:")

In [78]:
for dict in dict_of_dfs:
    print(dict) 

Teams
Players
Schedule
Player_Stats


In [79]:
dict_of_dfs['Teams'].head()

,Franchise,Lg,From,To,Yrs,G,W,L,W/L%,Plyfs,Div,Conf,Champ
0,Atlanta Hawks,NBA,1949-50,2023-24,75,5855,2891,2964,0.494,49,12,0,1
1,Boston Celtics,NBA/BAA,1946-47,2023-24,78,6032,3570,2462,0.592,60,33,10,17
2,Brooklyn Nets,NBA/ABA,1967-68,2023-24,57,4530,1996,2534,0.441,31,5,2,2
3,Charlotte Hornets,NBA,1988-89,2023-24,34,2631,1153,1478,0.438,10,0,0,0
4,Chicago Bulls,NBA,1966-67,2023-24,58,4598,2344,2254,0.510,37,9,6,6


In [80]:
for df in dict_of_dfs:
    dict_of_dfs[df].to_sql(df, con=engine)

In [81]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(
    engine,
    # include_tables=['Teams', 'Players_2023-24', 'Schedule_2023-24', 'Player_Stats_2022-23_(Playoffs)', 'Player_Stats_2022-23_(Regular_Season)']
    include_tables=list(dict_of_dfs.keys())
    )

In [82]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=list(dict_of_dfs.keys())
)

In [106]:
from llama_index.tools.query_engine import QueryEngineTool

sql_nba_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine, # 
    name='sql_nba_tool', 
    description=("""Useful for translating a natural language query into a SQL query over tables containing:
                    1. teams, containing information related to all NBA teams
                    2. players, containing information related to all NBA players
                    3. schedule, containing information related to the entire NBA game schedule
                    4. player_stats_playoffs, containing information related to all NBA player stats
                    """
    ),

)

In [116]:
from llama_index.agent import OpenAIAgent

agent = OpenAIAgent.from_tools(
    [sql_nba_tool, 
     auto_retrieve_tool_emails,
     auto_retrieve_tool_schedule], 
    llm=llm, verbose=True 
)

In [120]:
response = agent.chat("I am Stephen Curry. Do I have an email from Kevin Durant?")

=== Calling Function ===
Calling function: auto_retrieve_tool_emails with args: {
  "query": "from: Stephen Curry to: Kevin Durant",
  "filter_key_list": ["from", "to"],
  "filter_value_list": ["Stephen Curry", "Kevin Durant"]
}


ValueError: Expected where to have exactly one operator, got {'from': 'Stephen Curry', 'to': 'Kevin Durant'}

## SQL GSW Query Tool

In [88]:
sheet_names = ['GSW_Team_Roster', 'GSW_Players_Summary', 'GSW_staff_and_executives']
dict_of_dfs = {sheet: get_df_from_workbook(sheet) for sheet in sheet_names}
dict_of_dfs['GSW_Team_Roster'].head()

,No.,Player,Pos,Ht,Wt,Birth Date,Unnamed: 6,Exp,College
0,NaN,Klay Thompson,G-F,6-6,215,"February 8, 1990",us,10.0,Washington State
1,NaN,Jonathan Kuminga,F,6-8,210,"October 6, 2002",cd,2.0,NaN
2,NaN,Chris Paul,G,NaN,175,"May 6, 1985",us,18.0,Wake Forest
3,NaN,Stephen Curry,G,6-2,185,"March 14, 1988",us,14.0,Davidson
4,NaN,Moses Moody,G,6-6,205,"May 31, 2002",us,2.0,Arkansas
